In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
import cv2
from typing import List
from utils.homography import HomographySIFT, HomographyResult
import torch
from utils.constants import NORM_GL, DENORM_GL, U100C2C
from utils.image_stitcher import make_gif_of_sequence
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable


def load_files_from_dir(path):
    data = {}
    list_files = list(path.glob('*.npz'))
    for p in list_files:
        try:
            d = np.load(p)
        except:
            continue
        for k, v in d.items():
            data.setdefault(k, []).extend( v)
    if not data:
        try:
            data = np.load(path.with_suffix('.npz'))
        except:
            raise FileNotFoundError(f'No files found in {path}')
    indices = np.argsort(data['time_ns'])
    data = {k:np.stack(v)[indices] for k, v in data.items()}
    return data


path_to_files = Path('')

left = load_files_from_dir(path_to_files / 'pan')
right = load_files_from_dir(path_to_files / 'mono')

print(f"List of keys in dict: {list(right.keys())}\n")
print(f"Number of left frames: {len(left['time_ns']):,}")
print(f"Number of right frames: {len(right['time_ns']):,}")

Fix timing between stereo cameras

In [ ]:
MAX_TIME_DIFF = 0.0003  # seconds

def _get_diff(left, right):
    if len(left['time_ns']) > len(right['time_ns']):
        return np.abs(left['time_ns'][:, None] - right['time_ns'][None, :])
    else:
        return np.abs(left['time_ns'][None, :] - right['time_ns'][:, None])
diff_before = _get_diff(left, right).argmin(0)

# Remove frames that are not in both cameras
while abs(len(left['time_ns']) - len(right['time_ns'])) > 1:
    # Find the difference between the timestamps of the two cameras
    diff = _get_diff(left, right)

    # Find the indices in the longer array that have no counterpart in the shorter array
    diff_values_of_longer_dict = np.min(diff, axis=1)
    mask_longer_dict = diff_values_of_longer_dict <= (MAX_TIME_DIFF * 1e9)
    indices_of_longer_dict = np.where(mask_longer_dict)[0]
    if len(left['time_ns']) > len(right['time_ns']):
        left = {k: v[indices_of_longer_dict] for k, v in left.items()}
    else:
        right = {k: v[indices_of_longer_dict] for k, v in right.items()}
print(f"Length of left: {len(left['time_ns']):,}")
print(f"Length of right: {len(right['time_ns']):,}")

# Sort both arrays to have the closest timestamps have same index
diff = np.argmin(np.abs(left['time_ns'][:, None] - right['time_ns'][None, :]), axis=0)
left = {k: v[diff] for k, v in left.items()}

diff = np.abs(left['time_ns'] - right['time_ns'])*1e-9
print(f"Max time difference between frames: {np.max(diff):.2g} seconds")
print(f"Min time difference between frames: {np.min(diff):.2g} seconds")

plt.figure()
plt.scatter(range(len(left['time_ns'])), left['time_ns'] * 1e-9, label='Left', s=2)
plt.scatter(range(len(right['time_ns'])), right['time_ns'] * 1e-9, label='Right', s=2)
plt.title('Timestamps of frames')
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.scatter(range(len(diff)), diff, c='r', s=2)
plt.plot(np.ones_like(left['time_ns'])*np.mean(diff))
plt.grid()
plt.title('Time difference between frames')
plt.ylabel('Time difference [s]')
plt.show()

# diff = _get_diff(left, right)
# plt.figure()
# plt.plot(diff.argmin(0), label='After')
# plt.plot(diff_before, label='Before')
# plt.legend()
# plt.grid()
# plt.show()
# diff = left['time_ns'] - right['time_ns']


# Parameters

In [ ]:
distance_between_centers_of_cameras = 0.1  # meters   ?????????????????????????????????????????
distance_from_ground = 50 # meters
focal_length = 9.8 * 1e-3 # meters
size_of_pixel = 17 * 1e-6 # meters
image_width = 336 # pixels
image_height = 256 # pixels

delta_distance_from_ground = 3 # meters

In [ ]:
fov_width_rad = 2 * np.arctan(image_width * size_of_pixel / (2 * focal_length))
fov_width_degree = fov_width_rad * 180 / np.pi
fov_width_meters = 2 * distance_from_ground * np.tan(fov_width_rad / 2)
fov_height_rad = 2 * np.arctan(image_height * size_of_pixel / (2 * focal_length))
fov_height_degree = fov_height_rad * 180 / np.pi
fov_height_meters = 2 * distance_from_ground * np.tan(fov_height_rad / 2)
print(f"FOV width: {fov_width_degree:.1f}deg, {fov_width_meters:.1f}m")
print(f"FOV height: {fov_height_degree:.1f}deg, {fov_height_meters:.1f}m")


In [ ]:
fov_single_pixel_width_meters = 2 * distance_from_ground * size_of_pixel / focal_length
fov_single_pixel_height_meters = 2 * distance_from_ground * size_of_pixel / focal_length
print(f"FOV single pixel width: {fov_single_pixel_width_meters:.2g}m")
print(f"FOV single pixel height: {fov_single_pixel_height_meters:.2g}m")

min_fov_change = 2 * (distance_from_ground-delta_distance_from_ground) * size_of_pixel / focal_length
max_fov_change = 2 * (distance_from_ground+delta_distance_from_ground) * size_of_pixel / focal_length
print(f"Min FOV change: {min_fov_change:.2g}m")
print(f"Max FOV change: {max_fov_change:.2g}m")

In [ ]:
distance_between_cameras_in_pixels = distance_between_centers_of_cameras / fov_single_pixel_width_meters
print(f"Distance between cameras in pixels: {distance_between_cameras_in_pixels:.2g} pixels")

# An example of the data

In [ ]:
# plot the first frame side-by-side
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(left['frames'][0])
ax[1].imshow(right['frames'][0])
ax[0].set_title('Left camera')
ax[1].set_title('Right camera')
plt.show()

# Try finding the homography using cv2 on a single image

In [ ]:
# Use cv2 to find the homography between the frames

def find_homography(left, right):
    left = cv2.normalize(left, left, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    right = cv2.normalize(right, right, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(left, None)
    kp2, des2 = orb.detectAndCompute(right, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M, mask
find_homography(left['frames'][0], right['frames'][0])[0]

# Brute force search for the best homography
Results were not that good.

In [ ]:
# def warp_image(img, M):
#     return cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))

# def _calc_loss(M, left, right):
#     loss = 0
#     for idx in range(len(left['frames'])):
#         estimated_right = warp_image(left['frames'][idx], M)
#         loss += np.mean(np.abs((estimated_right - right['frames'][idx])))
#     loss /= len(left['frames'])
#     return loss


# NW = 100
# NH = 100
# array_distances_w = np.linspace(0.9, 2, NW)
# array_distances_h = np.linspace(5.5, 7, NH)
# M_permutations = np.eye(3)[None, None]
# M_permutations = M_permutations.repeat(NH, axis=0).repeat(NW, axis=1)
# M_permutations[:, :, 0, 2] = array_distances_h[:, None]
# M_permutations[:, :, 1, 2] = array_distances_w[None, :]
# M_permutations = M_permutations.reshape(-1, 3, 3)

# func = partial(_calc_loss, left=left.copy(), right=right.copy())


# with mp.Pool(mp.cpu_count()) as pool:
#     ret_val = list(tqdm(pool.imap(func, M_permutations, 
#                                   chunksize=2**6), 
#                                 #   chunksize=len(M_permutations) // mp.cpu_count()), 
#                                   total=len(M_permutations)))

# print(min(ret_val))
# print(M_permutations[np.argmin(ret_val)])
# plt.plot(ret_val)

# Brute force search using Kornia
Better results than cv2.

In [ ]:
DEVICE = 'cuda'

with torch.inference_mode():
    homography = HomographySIFT().to(DEVICE, dtype=torch.float32)
    x = np.concatenate([left['frames'][None], right['frames'][None]], axis=0).transpose(1, 0, 2, 3)
    x = NORM_GL(x.astype(float))
    x = torch.from_numpy(x)
    ret_val: List[HomographyResult] = [homography(x=x_.to(DEVICE, dtype=torch.float32), m=None, mask=None, verbose=False).cpu() for x_ in tqdm(torch.split(x, 1, dim=0))]
h = np.concatenate([p.homography for p in ret_val], 0)

### Plot the Kornia results

In [ ]:
for i, j in np.ndindex(3, 3):
    d = h[:, i, j]
    std = np.std(d)
    d = d[np.abs(d - np.mean(d)) < 1 * std]
    plt.figure()
    plt.scatter(range(len(d)), d, label='y', s=2)
    plt.plot(np.mean(d) * np.ones_like(d), label='mean', c='r', linewidth=1)
    # plt.plot(np.mean(d) * np.ones_like(d)-1,  '--', c='black', linewidth=1)
    # plt.plot(np.mean(d) * np.ones_like(d])+1,  '--', c='black', linewidth=1)
    # plt.yticks(np.arange(np.floor(d.min()), np.ceil(d.max()), 1))
    plt.title(f'Homography {i}, {j}')
    plt.grid()
    plt.show()
    plt.close()

In [ ]:
indices = np.arange(len(h))
np.random.shuffle(indices)
indices = indices[:10]
for idx in indices:
    static = U100C2C(DENORM_GL(ret_val[idx].static.squeeze()))
    dynamic = U100C2C(DENORM_GL(ret_val[idx].dynamic.squeeze()))
    warped = U100C2C(DENORM_GL(ret_val[idx].warped.squeeze()))
    mask = ret_val[idx].mask.squeeze().float()

    # Erode the mask to remove border artifacts
    SIZE_OF_EROSION = 9
    mask = torch.nn.functional.conv2d(
        mask[None, None],
        torch.ones(1, 1, SIZE_OF_EROSION, SIZE_OF_EROSION).to(mask),
        padding=SIZE_OF_EROSION // 2).squeeze()
    mask /= SIZE_OF_EROSION**2
    mask = mask == 1

    # plot the first frame side-by-side
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))
    ax[0].imshow(static)
    ax[0].set_title('Static')
    ax[1].imshow(dynamic)
    ax[1].set_title('Dynamic')
    ax[2].imshow(warped, vmin=warped[mask].min(), vmax=warped[mask].max())
    ax[2].set_title('Warped')

    # Calculate the difference between the static and the warped image
    diff = np.abs(static - warped)
    vmin = diff[mask].min()
    vmax = diff[mask].max()
    diff[~mask] = -1
    ax[3].imshow(diff, vmin=vmin, vmax=vmax)

    # Add colorbar to ax[3]
    divider = make_axes_locatable(ax[3])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(ax[3].imshow(diff, vmin=diff[mask].min(), vmax=diff[mask].max()), cax=cax)
    ax[3].set_title(f'Diff, avg err: {diff[mask].mean():.2g}C')

    plt.show()
    plt.close()

In [ ]:
path_to_save = Path('rawData')
path_to_save.mkdir(exist_ok=True)

for idx, frame in enumerate(left['frames']):
    np.save(path_to_save / f'left_{idx}.npy', frame)
for idx, frame in enumerate(right['frames']):
    np.save(path_to_save / f'right_{idx}.npy', frame)